In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from boardgame2 import ReversiEnv
from dynamic_programming import generate_uniform_stochastic_policy, policy_evaluation, stochastic_policy_eval_step, generate_deterministic_policy, deterministic_policy_eval_step
from tree_search import bfs_cannonical
import numpy as np

# Programación dinámica

En esta parte no es necesario la implementación de código ya que ya esta todo resuelto. Si tiene que responder algunas preguntas en **EDX**.

Si lo desea puede ver el código para analizar la implementación, pero es opcional

Si quiere profundizar le recomendamos mirar:

- bfs_cannonical cannonical de la librería tree_search
- policy_evaluation, policy_improve, policy_iterartion y value_iteration de dynamic_programming

### La idea de esta sección es generar las $V^*(s)$y $\Pi^*(s)$ (óptimas) en 4x4 para poder hacer los análisis posteriores
### Por eso se deben correr todas las celdas

# Busqueda de todos los estados canónicos

Solo desde el punto de vista del jugador +1

In [3]:
%%time
board_size = 4
states = bfs_cannonical(board_size, verbose=1)

Wall time: 1min 17s


Al ser canónico, no es necesario que el jugador sea parte del estado ya que siempre se puede pensar como que le toca jugar al jugador +1

In [4]:
# Listamos los primeros 5 estados encontrados
for s in list(states.keys())[0:5]:
    print(s)

(0, 0, 0, 0, 0, 1, -1, 0, 0, -1, 1, 0, 0, 0, 0, 0)
(0, 0, -1, 0, 0, -1, -1, 0, 0, 1, -1, 0, 0, 0, 0, 0)
(0, 0, 0, 0, 0, -1, -1, -1, 0, 1, -1, 0, 0, 0, 0, 0)
(0, 0, 0, 0, 0, -1, 1, 0, -1, -1, -1, 0, 0, 0, 0, 0)
(0, 0, 0, 0, 0, -1, 1, 0, 0, -1, -1, 0, 0, -1, 0, 0)


In [5]:
# Guardamos el estado s0
s0 = list(states.keys())[0]

In [6]:
s0

(0, 0, 0, 0, 0, 1, -1, 0, 0, -1, 1, 0, 0, 0, 0, 0)

In [7]:
# Mostrado como tablero
np.array(s0).reshape(4,4)

array([[ 0,  0,  0,  0],
       [ 0,  1, -1,  0],
       [ 0, -1,  1,  0],
       [ 0,  0,  0,  0]], dtype=int8)

Cada estado se guarda con todas sus posibles acciones y dado el estado y la acción, se guarda:
- **next_node**: el próximo estado al ejecutar esa acción
- **done**: si termina el juego (episodio)
- **winner**: si al ejecutar la acción alguno de los dos jugadores gana: (+1 o -1)

In [8]:
for action, next_data in states[s0].items():
    print(f'acción: {action}')
    print(next_data)

acción: (0, 2)
{'done': False, 'winner': -0.0, 'next_node': (0, 0, -1, 0, 0, -1, -1, 0, 0, 1, -1, 0, 0, 0, 0, 0)}
acción: (1, 3)
{'done': False, 'winner': -0.0, 'next_node': (0, 0, 0, 0, 0, -1, -1, -1, 0, 1, -1, 0, 0, 0, 0, 0)}
acción: (2, 0)
{'done': False, 'winner': -0.0, 'next_node': (0, 0, 0, 0, 0, -1, 1, 0, -1, -1, -1, 0, 0, 0, 0, 0)}
acción: (3, 1)
{'done': False, 'winner': -0.0, 'next_node': (0, 0, 0, 0, 0, -1, 1, 0, 0, -1, -1, 0, 0, -1, 0, 0)}


# Ejemplo de un estado terminal

In [9]:
done = 0
for s in states.keys():
    for action, next_data in states[s].items():
        if next_data['done']:
            print(s)
            print(f'acción: {action}')
            print(next_data)
            done = done + 1
            print()
            break
    if done > 5:
        break

(0, -1, -1, -1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0)
acción: (-1, 0)
{'done': True, 'winner': 1, 'next_node': (0, 1, 1, 1, 0, -1, -1, 0, -1, -1, -1, 0, 0, 0, 0, 0)}

(0, 0, 0, -1, 0, 1, 1, -1, 0, 1, 1, -1, 0, 1, 0, 0)
acción: (-1, 0)
{'done': True, 'winner': 1, 'next_node': (0, 0, 0, 1, 0, -1, -1, 1, 0, -1, -1, 1, 0, -1, 0, 0)}

(0, 0, 1, 0, -1, 1, 1, 0, -1, 1, 1, 0, -1, 0, 0, 0)
acción: (-1, 0)
{'done': True, 'winner': 1, 'next_node': (0, 0, -1, 0, 1, -1, -1, 0, 1, -1, -1, 0, 1, 0, 0, 0)}

(0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, -1, -1, -1, 0)
acción: (-1, 0)
{'done': True, 'winner': 1, 'next_node': (0, 0, 0, 0, 0, -1, -1, -1, 0, -1, -1, 0, 1, 1, 1, 0)}

(-1, -1, -1, 1, 0, -1, -1, 0, 0, 1, -1, -1, 0, 0, 0, 0)
acción: (-1, 0)
{'done': True, 'winner': 1, 'next_node': (1, 1, 1, -1, 0, 1, 1, 0, 0, -1, 1, 1, 0, 0, 0, 0)}

(-1, -1, -1, 0, -1, -1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0)
acción: (-1, 0)
{'done': True, 'winner': 1, 'next_node': (1, 1, 1, 0, 1, 1, -1, 0, 0, -1, -1, -1, 0, 0, 0, 0)}



La acción (-1, 0) es la acción PASS. En principio solo se ejecuta si no hay opciones válidas

In [10]:
states[(-1, -1, -1, 0, -1, -1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0)]

{(-1, 0): {'done': True,
  'winner': 1,
  'next_node': (1, 1, 1, 0, 1, 1, -1, 0, 0, -1, -1, -1, 0, 0, 0, 0)}}

# Policy Evaluation

### Politica estocástica

In [11]:
stochastic_pi = generate_uniform_stochastic_policy(states)

In [12]:
# ejemplos
stochastic_pi[(0, 0, 0, 0, 0, 1, -1, 0, 0, -1, 1, 0, 0, 0, 0, 0)]

{(0, 2): 0.25, (1, 3): 0.25, (2, 0): 0.25, (3, 1): 0.25}

In [13]:
stochastic_pi[(0, 0, -1, 0, 0, -1, -1, 0, 0, 1, -1, 0, 0, 0, 0, 0)]

{(0, 1): 0.3333333333333333,
 (0, 3): 0.3333333333333333,
 (2, 3): 0.3333333333333333}

Esto genera una política con distribución uniforme que luego será evaluada usuando **policy evaluation**

In [14]:
V_stochastic, iters = policy_evaluation(stochastic_policy_eval_step, 
                             states, 
                             stochastic_pi, 1e-8, verbose=1)

Iteration number:  1 2 3 4 5 6 7 8 9 10 11 12 13 


#### Ejemplos de la V luego de converger

In [15]:
V_stochastic[(0, 0, 0, 0, 0, 1, -1, 0, 0, -1, 1, 0, 0, 0, 0, 0)]

-0.2403001935859148

In [16]:
V_stochastic[(0, 0, -1, 0, 0, -1, -1, 0, 0, 1, -1, 0, 0, 0, 0, 0)]

0.2403001935859148

### Política determinística

In [17]:
det_pi = generate_deterministic_policy(states)

In [18]:
det_pi[(0, 0, 0, 0, 0, 1, -1, 0, 0, -1, 1, 0, 0, 0, 0, 0)]

(1, 3)

In [19]:
det_pi[(0, 0, -1, 0, 0, -1, -1, 0, 0, 1, -1, 0, 0, 0, 0, 0)]

(2, 3)

Notar que ahora la política dado el estado tiene solo una acción posible que se construyó de manera arbitraria

In [20]:
# Run it multiple times to check it takes different number of iterations to converge
V_det, _ = policy_evaluation(deterministic_policy_eval_step, 
                             states, 
                             det_pi, 1e-8, verbose=1)

Iteration number:  1 2 3 4 5 6 7 8 9 10 11 12 


#### Ejemplos de la V luego de converger

In [21]:
V_det[(0, 0, 0, 0, 0, 1, -1, 0, 0, -1, 1, 0, 0, 0, 0, 0)]

-1

In [22]:
V_det[(0, 0, -1, 0, 0, -1, -1, 0, 0, 1, -1, 0, 0, 0, 0, 0)]

1

# Policy Iteration

Partiendo de cualquier política (estocástica o determinística), por medio de Policy Iteration se puede obtener las óptimas 

In [23]:
from dynamic_programming import policy_improve, policy_iteration, generate_deterministic_policy

In [24]:
%%time
initial_policy = generate_deterministic_policy(states)
optimum_policy, optimum_V = policy_iteration(states, initial_policy, verbose = 1)

Iteration number:  1 2 3 4 5 6 7 8 9 10 11 12 13 
Number of differences of new policy vs old policy: 12606
---------------------------
Iteration number:  1 2 3 4 5 6 7 8 9 10 11 12 
Number of differences of new policy vs old policy: 1973
---------------------------
Iteration number:  1 2 3 4 5 6 7 8 9 10 11 
Number of differences of new policy vs old policy: 523
---------------------------
Iteration number:  1 2 3 4 5 6 7 8 9 10 11 12 13 
Number of differences of new policy vs old policy: 119
---------------------------
Iteration number:  1 2 3 4 5 6 7 8 9 10 11 12 13 
Number of differences of new policy vs old policy: 27
---------------------------
Iteration number:  1 2 3 4 5 6 7 8 9 10 11 12 13 
Number of differences of new policy vs old policy: 9
---------------------------
Iteration number:  1 2 3 4 5 6 7 8 9 10 11 12 13 
Number of differences of new policy vs old policy: 3
---------------------------
Iteration number:  1 2 3 4 5 6 7 8 9 10 11 12 13 
Number of differences of new p

In [26]:
np.save('mdp/pi_mdp', optimum_policy)
np.save('mdp/v_mdp', optimum_V)

In [27]:
optimum_policy[(0, 0, -1, 0, 0, -1, -1, 0, 0, 1, -1, 0, 0, 0, 0, 0)]

(0, 3)

In [28]:
optimum_V[(0, 0, -1, 0, 0, -1, -1, 0, 0, 1, -1, 0, 0, 0, 0, 0)]

1

In [29]:
np.array((0, 0, -1, 0, 0, -1, -1, 0, 0, 1, -1, 0, 0, 0, 0, 0)).reshape(4, 4)

array([[ 0,  0, -1,  0],
       [ 0, -1, -1,  0],
       [ 0,  1, -1,  0],
       [ 0,  0,  0,  0]])

# Value Iteration

In [30]:
from dynamic_programming import value_iteration

In [31]:
%%time
V, delta = value_iteration(states, verbose=1)

1 16 2.148329015302604
2 14 1.3984082309742596
3 14 0.7103688654451921
4 13 0.3661814318465639
5 12 0.1380402974781458
6 11 0.05770628692848223
7 10 0.02005554416506682
8 8 0.006710033363777003
9 6 0.0023857896404540454
10 6 0.0005964474101135114
11 6 0.00011183388939628339
12 0 0.0
Wall time: 6.08 s


# Programación Dinámica Cuestionario

## Respuesta a los ejercicios de laboratorio

### 1)

In [32]:
s1 = (1, 1, 1, -1, 0, 1, 1, 0, 0, -1, 1, 1, 0, 0, 0, 0)

In [33]:
states[s1]

{(2, 0): {'done': False,
  'winner': -0.0,
  'next_node': (-1, -1, -1, 1, 0, -1, -1, 0, -1, -1, -1, -1, 0, 0, 0, 0)},
 (3, 0): {'done': True,
  'winner': -1,
  'next_node': (-1, -1, -1, 1, 0, -1, -1, 0, 0, -1, -1, -1, -1, 0, 0, 0)},
 (3, 1): {'done': False,
  'winner': -0.0,
  'next_node': (-1, -1, -1, 1, 0, -1, -1, 0, 0, -1, -1, -1, 0, -1, 0, 0)}}

In [34]:
states[s1][(3,0)]['done']

True

In [35]:
states[s1][(3,0)]['winner']

-1

In [36]:
states[s1][(3,0)]['next_node']

(-1, -1, -1, 1, 0, -1, -1, 0, 0, -1, -1, -1, -1, 0, 0, 0)

### 2)

In [37]:
stochastic_pi[(0, 0, -1, 0, 0, -1, -1, 0, 0, 1, -1, 0, 0, 0, 0, 0)]

{(0, 1): 0.3333333333333333,
 (0, 3): 0.3333333333333333,
 (2, 3): 0.3333333333333333}

In [38]:
stochastic_pi[(0, 0, -1, 0, 0, -1, -1, 0, 0, 1, -1, 0, 0, 0, 0, 0)][(0,3)]

0.3333333333333333

La accion (3,3) no forma parte de las acciones posibles por lo tanto la probabilidad es 0

### 3)

La evaluacion de la politica estocastica pondera en el calculo de V la probabilidad de pasar a el nuevo estado V[siguiente estado] 
que puede ser un numero menor a 1 con lo cual en el calculo el V puede dar un numero que no sea entero

In [39]:
V_stochastic[(0, 0, -1, 0, 0, -1, -1, 0, 0, 1, -1, 0, 0, 0, 0, 0)]

0.2403001935859148

Para el caso de la politica deterministica es distinto por que la probabilidad de tomar una accion y pasar al siguiente estado es siempre 1 (entorno deterministico)

In [40]:
V_det[(0, 0, -1, 0, 0, -1, -1, 0, 0, 1, -1, 0, 0, 0, 0, 0)]

1

### 4)

In [41]:
optimum_V[(0, 0, -1, 0, 0, -1, -1, 0, 0, 1, -1, 0, 0, 0, 0, 0)]

1

In [42]:
optimum_policy[(0, 0, -1, 0, 0, -1, -1, 0, 0, 1, -1, 0, 0, 0, 0, 0)]

(0, 3)

### 5)

In [43]:
s0 = (0, 0, 0, 0, 0, 1, -1, 0, 0, -1, 1, 0, 0, 0, 0, 0)

In [44]:
np.array(s0).reshape(4,4)

array([[ 0,  0,  0,  0],
       [ 0,  1, -1,  0],
       [ 0, -1,  1,  0],
       [ 0,  0,  0,  0]])

In [45]:
optimum_V[s0]

-1

El primero no puede lograr un empate, se supone que V es el value considerando la mejor respuesta

In [ ]:
optimum_policy

In [47]:
s1 = (0,  0,  -1,  0, 0,  -1,  -1,  0, 0, 1,  -1,  0, 0,  0,  0,  0)

In [48]:
#s1 = (0, 0, 0, 0, 0, -1, 1, 0, -1, -1, -1, 0, 0, 0, 0, 0)

In [49]:
optimum_V[s1]

1

In [50]:
optimum_policy[s1]

(0, 3)

In [51]:
np.array(s1).reshape(4,4)

array([[ 0,  0, -1,  0],
       [ 0, -1, -1,  0],
       [ 0,  1, -1,  0],
       [ 0,  0,  0,  0]])

In [52]:
s2 = (0,  -1, 1,  0, 0, -1, 1,  0, 0,  -1, 1,  0, 0,  0,  0,  0) #el segundo jugando mal en su primer turno

In [53]:
np.array(s2).reshape(4,4)

array([[ 0, -1,  1,  0],
       [ 0, -1,  1,  0],
       [ 0, -1,  1,  0],
       [ 0,  0,  0,  0]])

In [54]:
optimum_V[s2]

1

Por lo tanto se ve que el segundo pierde cuando juega mal su primer turno ya que la V es 1

In [55]:
s2 = (0,  0, 1,  -1, 0, 1, -1,  0, 0,  -1, 1,  0, 0,  0,  0,  0)  #el segundo jugando bien en su primer turno

In [56]:
np.array(s2).reshape(4,4)

array([[ 0,  0,  1, -1],
       [ 0,  1, -1,  0],
       [ 0, -1,  1,  0],
       [ 0,  0,  0,  0]])

In [57]:
optimum_V[s2]

-1

In [58]:
optimum_policy[s2]

(1, 3)

In [59]:
s2 = np.array(s2)

In [60]:
s2[1*4+3] = 1
s2[1*4+2] = 1


In [61]:
s2 = s2 * -1

In [62]:
np.array(s2).reshape(4,4)

array([[ 0,  0, -1,  1],
       [ 0, -1, -1, -1],
       [ 0,  1, -1,  0],
       [ 0,  0,  0,  0]])

In [63]:
s2 = tuple(s2)

In [64]:
optimum_V[s2]

1

In [65]:
optimum_policy[s2]

(0, 1)

In [66]:
s2 = (0,  -1, -1, -1, 0, -1, 1, 1, 0,  -1, 1,  0, 0,  0,  0,  0) 

In [67]:
np.array(s2).reshape(4,4)

array([[ 0, -1, -1, -1],
       [ 0, -1,  1,  1],
       [ 0, -1,  1,  0],
       [ 0,  0,  0,  0]])

In [68]:
optimum_policy[s2]

(0, 0)

In [69]:
optimum_V[s2]

-1

In [70]:
s2 = (-1,  1, 1, 1, 0, -1, -1, -1, 0,  1, -1,  0, 0,  0,  0,  0) 

In [71]:
optimum_policy[s2]

(2, 3)

In [72]:
np.array(s2).reshape(4,4)

array([[-1,  1,  1,  1],
       [ 0, -1, -1, -1],
       [ 0,  1, -1,  0],
       [ 0,  0,  0,  0]])

In [73]:
s2 = (1, -1, -1, -1, 0, 1, -1, -1, 0, -1, -1, -1, 0, 0, 0, 0) 

In [74]:
optimum_policy[s2]

(3, 1)

In [75]:
np.array(s2).reshape(4,4)

array([[ 1, -1, -1, -1],
       [ 0,  1, -1, -1],
       [ 0, -1, -1, -1],
       [ 0,  0,  0,  0]])

Se observa que el jugador 2 siempre come en diagonal